In [1]:
!pip install faiss-cpu sentence-transformers

In [ ]:
import pandas as pd
import faiss
import numpy as np
import ast
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import torch

# =============================================================================
# 1. 설정 (Configuration)
# =============================================================================
# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"현재 사용 디바이스: {device}")

# 모델 설정 (BGE-m3-ko: 긴 문맥 처리에 강함, 벡터 차원 1024)
MODEL_NAME = 'dragonkue/BGE-m3-ko'
BATCH_SIZE = 16  # GPU 메모리에 따라 조절 (OOM 발생 시 줄이세요)
TOP_K = 3        # 각 질문당 찾을 유사 문서 개수

# =============================================================================
# 2. 모델 로드
# =============================================================================
print(f"모델 로드 중... ({MODEL_NAME})")
model = SentenceTransformer(MODEL_NAME, device=device)

# BGE-m3의 벡터 차원 가져오기 (1024 예상)
dimension = model.get_sentence_embedding_dimension()
print(f"모델 벡터 차원: {dimension}")

# =============================================================================
# 3. Query 데이터 (CSV) 로드 및 전처리
# =============================================================================
print("\n[Step 1] CSV 데이터 로드 및 필터링 중...")
try:
    df_csv = pd.read_csv("/data/ephemeral/pro-nlp-generationfornlp-nlp-13/data/test.csv")
except FileNotFoundError:
    print("오류: 'test.csv' 파일을 찾을 수 없습니다.")
    exit()

query_texts = []   # 검색할 질문(question) 리스트
query_ids = []     # 원본 ID 리스트

for idx, row in df_csv.iterrows():
    try:
        # 문자열로 된 딕셔너리 파싱 ("{'question': ...}")
        # 데이터가 비어있거나 nan인 경우 방지
        if pd.isna(row['problems']):
            continue

        problem_data = ast.literal_eval(row['problems'])

        # 필터링 조건: 'choices'가 있고 개수가 4개인 경우
        if 'choices' in problem_data and isinstance(problem_data['choices'], list):
            if len(problem_data['choices']) == 4:
                # 검색어: 질문(question) 추출
                if 'question' in problem_data:
                    query_texts.append(problem_data['question'])
                    query_ids.append(row['id'])

    except (ValueError, SyntaxError):
        # 파싱 에러 발생 시 해당 행 건너뜀
        continue

print(f" -> 원본 데이터 수: {len(df_csv)}")
print(f" -> 필터링(choices=4) 후 검색 대상 수: {len(query_texts)}")

if len(query_texts) == 0:
    print("검색할 데이터가 없습니다. 조건을 확인해주세요.")
    exit()

# =============================================================================
# 4. Base 데이터 (위키피디아) 로드 및 FAISS 인덱싱
# =============================================================================
print("\n[Step 2] 위키피디아 데이터 로드 및 인덱싱...")

# 테스트를 위해 상위 5만 건만 로드 (전체를 하려면 split='train'으로 변경)
# 주의: 전체 데이터(수백만 건) 로드 시 시간이 오래 걸립니다.
ds_wiki = load_dataset("lcw99/wikipedia-korean-20240501", split="train[:50000]")

# FAISS 인덱스 생성 (Inner Product = Cosine Similarity용)
index = faiss.IndexFlatIP(dimension)

# 배치 단위로 임베딩 후 인덱스에 추가
total_wiki = len(ds_wiki)

for i in tqdm(range(0, total_wiki, BATCH_SIZE), desc="Wiki Indexing"):
    # 배치 슬라이싱
    batch_data = ds_wiki[i : i + BATCH_SIZE]
    batch_texts = batch_data['text'] # 위키 본문

    # 임베딩 생성 (normalize_embeddings=True 필수 for 코사인 유사도)
    embeddings = model.encode(batch_texts, batch_size=BATCH_SIZE, normalize_embeddings=True, show_progress_bar=False)

    # FAISS 추가
    index.add(embeddings.astype('float32'))

print(f" -> 인덱싱 완료. 총 문서 수: {index.ntotal}")

# =============================================================================
# 5. 유사도 검색 수행
# =============================================================================
print("\n[Step 3] 질문(Query) 검색 수행 중...")

# 쿼리 데이터 임베딩
query_embeddings = model.encode(query_texts, batch_size=BATCH_SIZE, normalize_embeddings=True, show_progress_bar=True)

# 검색 (Top K)
distances, indices = index.search(query_embeddings.astype('float32'), TOP_K)

import os
import matplotlib.pyplot as plt

# =============================================================================
# [Server Config] 서버 환경 설정
# =============================================================================
# 서버에서는 창을 띄울 수 없으므로 그래프 백엔드를 'Agg'로 설정 (에러 방지 필수)
plt.switch_backend('Agg')

# 결과물을 저장할 폴더 지정 (현재 실행 위치의 'server_results' 폴더)
SAVE_DIR = "./server_results"
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"\n📂 [Server Setup] 결과 저장 경로: {os.path.abspath(SAVE_DIR)}")

# =============================================================================
# [Step 6] 결과 매칭 및 데이터프레임 생성
# =============================================================================
print("\n[Step 6] 검색 결과 매칭 중...")

results = []

# indices와 distances는 앞선 faiss.search()의 결과값입니다.
for i in range(len(query_texts)):
    original_id = query_ids[i]
    q_text = query_texts[i]
    
    for rank in range(TOP_K):
        found_idx = indices[i][rank]
        score = distances[i][rank]
        
        # 유효하지 않은 인덱스 제외
        if found_idx == -1:
            continue
            
        # 위키피디아 데이터 가져오기 (ds_wiki는 메모리에 로드된 상태여야 함)
        wiki_item = ds_wiki[int(found_idx)]
        
        results.append({
            'query_id': original_id,
            'question_text': q_text,
            'rank': rank + 1,
            'similarity_score': score, 
            'wiki_title': wiki_item.get('title', ''),
            'wiki_text': wiki_item.get('text', '')[:1000] # 본문 1000자까지 저장
        })

# DataFrame 변환
df_result = pd.DataFrame(results)

# =============================================================================
# [Step 7] 결과 CSV 저장
# =============================================================================
csv_filename = "final_similarity_results.csv"
csv_path = os.path.join(SAVE_DIR, csv_filename)

# 한글 깨짐 방지를 위해 utf-8-sig 사용
df_result.to_csv(csv_path, index=False, encoding='utf-8-sig')
print(f"✅ CSV 파일 저장 완료: {csv_path}")

# =============================================================================
# [Step 8] 데이터셋 전체 품질 분석 리포트 (서버 로그용)
# =============================================================================
print("\n" + "="*50)
print("📊 [유사도 분석 리포트]")
print("="*50)

# Rank 1(가장 적절한 답변)만 추출하여 통계 산출
best_matches = df_result[df_result['rank'] == 1].copy()

if not best_matches.empty:
    mean_score = best_matches['similarity_score'].mean()
    median_score = best_matches['similarity_score'].median()
    max_score = best_matches['similarity_score'].max()
    min_score = best_matches['similarity_score'].min()

    # 점수 구간별 개수 계산
    def classify_score(score):
        if score >= 0.85: return 'Very High (0.85+)'
        elif score >= 0.75: return 'High (0.75~0.84)'
        elif score >= 0.65: return 'Medium (0.65~0.74)'
        else: return 'Low (<0.65)'

    best_matches['quality'] = best_matches['similarity_score'].apply(classify_score)
    quality_counts = best_matches['quality'].value_counts()

    print(f"• 분석 대상 질문 수 : {len(best_matches)} 개")
    print(f"• 평균 유사도       : {mean_score:.4f}")
    print(f"• 중간값(Median)    : {median_score:.4f}")
    print(f"• 최고 / 최저 점수  : {max_score:.4f} / {min_score:.4f}")
    print("-" * 50)
    print("[점수대별 분포]")
    print(quality_counts.to_string())
    
    # ---------------------------------------------------------
    # [Step 9] 그래프 이미지 저장 (서버용)
    # ---------------------------------------------------------
    try:
        plt.figure(figsize=(10, 6))
        plt.hist(best_matches['similarity_score'], bins=30, color='#4c72b0', edgecolor='black', alpha=0.7)
        plt.title(f'Similarity Score Distribution (Mean: {mean_score:.2f})')
        plt.xlabel('Cosine Similarity')
        plt.ylabel('Count')
        plt.axvline(mean_score, color='red', linestyle='dashed', linewidth=1, label='Mean')
        plt.legend()
        plt.grid(axis='y', alpha=0.3)
        
        # 파일로 저장 (show() 호출 금지)
        img_filename = "similarity_distribution.png"
        img_path = os.path.join(SAVE_DIR, img_filename)
        plt.savefig(img_path)
        plt.close() # 메모리 해제
        
        print(f"✅ 그래프 이미지 저장 완료: {img_path}")
        
    except Exception as e:
        print(f"⚠️ 그래프 저장 중 오류 발생 (데이터 저장에는 영향 없음): {e}")

else:
    print("⚠️ 결과 데이터가 없습니다. 매칭된 문서가 하나도 없습니다.")

print("="*50)
print("모든 작업이 완료되었습니다.")

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /opt/conda/lib/python3.10/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


현재 사용 디바이스: cuda
모델 로드 중... (dragonkue/BGE-m3-ko)
모델 벡터 차원: 1024

[Step 1] CSV 데이터 로드 및 필터링 중...
 -> 원본 데이터 수: 869
 -> 필터링(choices=4) 후 검색 대상 수: 163

[Step 2] 위키피디아 데이터 로드 및 인덱싱...


Wiki Indexing:   0%|          | 0/782 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 GiB. GPU 0 has a total capacity of 31.74 GiB of which 11.25 GiB is free. Process 2708792 has 20.49 GiB memory in use. Of the allocated memory 20.12 GiB is allocated by PyTorch, and 12.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)